In [266]:
# coding=utf-8
import sys
import numpy as np
import requests
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile
from ast import literal_eval

# Константы
all_words_links_path = 'all_words_links2.npy'
vsekidki_fullwords_csv = 'vsekidki_fullwords.csv'
vsekidki_appwords_csv = "vsekidki_appwords.csv"
vsekidki_words_page = 'vsekidki_words_npy/vsekidki_words_page'
id_offset = 20000
global length
length = 55

## Сделать all_words_links = [ссылки на все слова] (загрузить с диска)

In [264]:
num_of_slang_words_pages = 82

In [267]:
all_words_links_path = 'cat_to_links'

In [370]:
if os.path.isfile(all_words_links_path):
    cat_to_links = np.load(all_words_links_path).tolist()
else:
    vsekidki_url = "http://vsekidki.ru/page/"
    cat_to_links = dict()
    link_to_cats = dict()
    for i in range(1, num_of_slang_words_pages):
        print(i, end=' ')
        r = requests.get(vsekidki_url + str(i) + '/')
        data = r.text
        soup = BeautifulSoup(data, "lxml")
        divs = soup.find_all('div', id="wid_m_bc")
        for div in divs:
            link = div.find('a')['href']
            cats = div.find('div', id='wid_m_cat').text
            cats = list(map(lambda x: x.strip(), filter(lambda x: x, cats.split('  '))))
            link = link.replace('https://vsekidki.ru/', '')
            link_to_cats[link] = cats
            for cat in cats:
                if not cat in cat_to_links:
                    cat_to_links[cat] = []
                cat_to_links[cat].append(link)
    if len(cat_to_links) > 0:
        print(len(cat_to_links))
        np.save(all_words_links_path, cat_to_links)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 36


In [371]:
cat_to_links.keys()

dict_keys(['Модный словарь', 'Геймеры', 'Музыка', 'Промокоды', 'В жизни', 'Фанаты', 'Модники', 'Интернет', 'Наркоманы', 'Иностранные слова/Перевод', 'Тюрьма', 'Гопники', 'Виды спорта', 'Аниме', 'Студенты', 'Армейский', 'Сложный прекол!', 'Узнать значение слова', 'Новые песни', 'Интернет песни', 'Блог о моде и стиле', 'Инфа сотка', 'On tour', 'В городе', 'Движи', 'Отзывы о фильмах', 'Слухи и информация', 'Социальные сети', 'Вязание', 'Коллекции', 'Мода', 'Look At Me все кидки', 'Полученная халява', 'Равнинный трюк', 'Флип трюк', 'Места'])

In [372]:
len(cat_to_links['Модный словарь'])

1517

In [375]:
link_to_cats['9764-deyliki.html']

['Модный словарь', 'Геймеры']

In [376]:
unique_links = set()
for cat in cat_to_links:
    unique_links.update(cat_to_links[cat])
len(unique_links)

1944

## Основные функции

In [377]:
def find_name(soup):
    return soup.find('h1').b.text.replace('Что такое ', '').rstrip('?')

def find_definition(soup):
    definition = soup.find('div', itemprop="text").contents
    return '\n '.join(filter(lambda x: x and not x.startswith('<'), map(lambda x: str(x).strip(' \n'), definition)))

def find_definition1(soup):
    text = []
    for x in soup.find('div', itemprop="text").contents:
        try:
            string = BeautifulSoup(str(x).replace('<br/>', '\n')).text
        except:
            string = x
        string = string.replace('(adsbygoogle = window.adsbygoogle || []).push({});', '')
        string = string.strip(' \n')
        text.append(string)
    text = list(filter(lambda x: x, text))
    return '\n'.join(text)

def sort_df_by_name(df):
    return df.sort_values('name', ascending=True).reset_index(drop=True)

def delete_rows_where_name_contains(string, df):
    indexes = []
    for index, row in df.iterrows():
        if row['name'].find("(Требуется значение слова)") != -1:
            indexes.append(index)
    print(indexes)
    #df = df.drop(df.index[indexes], inplace=True)
    #df = sort_df_by_name(df)
    return df

def save_words_csv(all_words, file):
    words_df = pd.DataFrame(all_words)
    words_df = words_df[['name', 'definition']]
    words_df.replace('', np.nan, inplace=True)
    words_df.dropna(inplace=True)
    words_df = sort_df_by_name(words_df)
    words_df.to_csv(file, index=False)
    return words_df

## Распарсить pages from 1 to length, поместить их в vsekidki_words_page(i).npy
#### Ничего не делать, если файлы есть.

In [298]:
with open ('unique_words.txt', 'w') as links_file:
    links_file.write('\n'.join(unique_links))

In [299]:
!aria2c -i {'unique_words.txt'} -d words_pages


04/15 17:57:00 [NOTICE] Downloading 1944 item(s)
[DL:336KiB][#076333 0B/0B][#bccc40 31KiB/0B][#e3a461 31KiB/0B][#8a7771 31KiB/0B#accd6b 0B/
04/15 17:57:03 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/483-gotov-ebalo.html

04/15 17:57:04 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/670-lol-smeh.html

04/15 17:57:04 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/9694-trial.html

04/15 17:57:04 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/9530-padal.html
[DL:208KiB][#076333 31KiB/0B][#7b5fe1 0B/0B][#7927c1 0B/0B][#5f10a2 0B/0B][#f0f
04/15 17:57:04 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1207-foreva.html
[DL:219KiB][#7b5fe1 47KiB/0B][#7927c1 31KiB/0B][#5f10a2 47KiB/0B][#f0f1f6 47KiB
04/15 17:57:05 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vseki

[DL:262KiB][#8b5c38 47KiB/0B][#d3da29 47KiB/0B][#000690 31KiB/0B][#9c8a9e 0B/0B
04/15 17:57:14 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/4213-basket.html

04/15 17:57:14 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1264-translit.html

04/15 17:57:14 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1480-kaputino.html

04/15 17:57:14 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/699-friendzone.html
[DL:226KiB][#abf457 0B/0B][#68f7fe 0B/0B][#d78ccb 15KiB/0B][#96026a 0B/0B][#b6f
04/15 17:57:15 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/241-backside-180.html

04/15 17:57:16 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/598-hvost-hvosty.html
[DL:133KiB][#abf457 0B/0B][#68f7fe 0B/0B][#b6fe8b 0B/0B][#de5d66 63KiB/0B][#99c
04/15 17:57:17 [NOTICE] D


04/15 17:57:43 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/881-akkaunt.html
[DL:108KiB][#d1273b 47KiB/0B][#c4ab12 0B/0B][#b31a40 0B/0B][#2408f9 47KiB/0B][#
04/15 17:57:44 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/959-dyup.html

04/15 17:57:44 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/675-mikro-mikro-kontrol.html

04/15 17:57:44 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/722-alesha.html

04/15 17:57:45 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1379-duck-army-armiya-utok-utinaya-armiya-utinyy-hor.html
[DL:117KiB][#d1273b 47KiB/0B][#c4ab12 0B/0B][#8b2057 0B/0B][#d6a8cf 0B/0B][#100
04/15 17:57:45 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/734-battl.html

04/15 17:57:45 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangA

[DL:123KiB][#553927 31KiB/0B][#beb094 31KiB/0B][#e91972 0B/0B][#544dda 0B/0B][#
04/15 17:58:25 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/9549-groak.html

04/15 17:58:25 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/4454-ptica-zlica.html

04/15 17:58:25 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/76-eto-vtoroy-vopros.html

04/15 17:58:26 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/276-v-ikeyu-za-mitbolami.html
[DL:123KiB][#45f302 0B/0B][#6b8c92 0B/0B][#13d7ef 0B/0B][#598dfa 0B/0B][#cc2fed
04/15 17:58:27 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/2874-ficher.html
[DL:112KiB][#45f302 47KiB/0B][#13d7ef 0B/0B][#598dfa 0B/0B][#cc2fed 0B/0B][#240
04/15 17:58:27 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/5452-come-on.html

04/15 17:58:28 

[DL:108KiB][#ab698c 0B/0B][#38a15f 0B/0B][#b1cc03 15KiB/0B][#467e7a 0B/0B][#0c3
04/15 17:58:48 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1149-80-lvl.html
[DL:109KiB][#ab698c 47KiB/0B][#38a15f 31KiB/0B][#467e7a 31KiB/0B][#0c3218 0B/0B[#9d3
04/15 17:58:50 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1197-mdek.html
[DL:108KiB][#ab698c 47KiB/0B][#38a15f 47KiB/0B][#467e7a 31KiB/0B][#9d3786 31KiB
04/15 17:58:51 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1922-zhdun.html

04/15 17:58:51 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/313-epicheskie-tuzy-gosha-rubchinskiy.html

04/15 17:58:51 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/4340-1000-rubley-na-bronirovanie-cherez-booking.html

04/15 17:58:51 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vseki

...

[DL:142KiB][#29e605 47KiB/0B][#0c2b00 47KiB/0B][#587b3e 15KiB/0B][#391b69 0B/0B[#c34
04/15 18:08:37 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1390-dnyuha-denyuha.html

04/15 18:08:38 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/786-bombing.html
[DL:136KiB][#29e605 47KiB/0B][#0c2b00 47KiB/0B][#c34c2a 0B/0B][#fadc81 0B/0B][#
04/15 18:08:38 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/549-esad.html

04/15 18:08:38 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/661-hp-hit-points.html
[DL:124KiB][#29e605 47KiB/0B][#fadc81 31KiB/0B][#a71926 0B/0B][#b57951 0B/0B][#
04/15 18:08:39 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/9585-flet.html

04/15 18:08:40 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/578-haypovyy-hayp-haypim.html
[DL:115KiB][#29e


04/15 18:09:00 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1750-enemi.html
[DL:114KiB][#12645e 15KiB/0B][#1c0148 15KiB/0B][#01e85a 0B/0B][#f6e269 0B/0B][#
04/15 18:09:01 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1073-knyupel.html

04/15 18:09:01 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/4819-opay-yo.html
[DL:114KiB][#12645e 31KiB/0B][#f6e269 0B/0B][#1a602f 0B/0B][#3a3fe3 0B/0B][#736
04/15 18:09:02 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/696-zadenait.html
[DL:112KiB][#12645e 31KiB/0B][#1a602f 31KiB/0B][#3a3fe3 0B/0B][#736ded 0B/0B][#
04/15 18:09:02 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/938-nerf.html

04/15 18:09:03 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1042-taz-tazodrocher.html

04/15 18:09:03 [NOTICE] Download c

[DL:118KiB][#353cae 15KiB/0B][#7e1bd5 0B/0B][#54255c 0B/0B][#d98406 0B/0B][#d3a
04/15 18:09:16 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/853-koresh.html
[DL:120KiB][#7e1bd5 47KiB/0B][#54255c 15KiB/0B][#d98406 0B/0B][#d3a643 0B/0B][#
04/15 18:09:17 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/854-banka.html
[DL:115KiB][#7e1bd5 47KiB/0B][#d98406 0B/0B][#d3a643 31KiB/0B][#f1d2b3 0B/0B][#
04/15 18:09:17 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/3963-chd.html

04/15 18:09:18 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/673-makro-makro-kontrol.html
[DL:108KiB][#7e1bd5 47KiB/0B][#d98406 0B/0B][#b934c2 15KiB/0B][#f5efc6 0B/0B][#
04/15 18:09:19 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/889-bild.html

04/15 18:09:19 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/Sla

[DL:104KiB][#2a2e85 47KiB/0B][#b8ae50 15KiB/0B][#9a44cf 0B/0B][#6cdc53 0B/0B][#
04/15 18:09:35 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/3517-gruming.html
[DL:104KiB][#2a2e85 47KiB/0B][#9a44cf 47KiB/0B][#6cdc53 15KiB/0B][#3f75aa 0B/0B
04/15 18:09:36 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/780-kudere.html

04/15 18:09:37 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/9656-kosyachit.html
[DL:106KiB][#2a2e85 47KiB/0B][#9a44cf 47KiB/0B][#dd5477 15KiB/0B][#d2c47b 0B/0B
04/15 18:09:38 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/844-slon.html

04/15 18:09:38 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/3525-travlya.html
[DL:105KiB][#2a2e85 47KiB/0B][#9a44cf 47KiB/0B][#48a802 0B/0B][#95e692 0B/0B][#
04/15 18:09:39 [NOTICE] Download complete: /Users/lotfull/Desktop/RuSlangApp/Slang

2d3bca|OK  |    62KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/219-zhiza.html
0c6ba9|OK  |    54KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/9754-oldy.html
5e0542|OK  |    51KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/855-ployka.html
25d229|OK  |    50KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/562-322.html
fc031e|OK  |    49KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/418-schemit-ebalo.html
2ebce4|OK  |    48KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/1132-dvizh-dvizhuha.html
430dc1|OK  |    38KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/860-gayd.html
92b622|OK  |    38KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/5381-bent.html
801f56|OK  |    39KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki/297-premiya-oskar-2016-onlayn.html
5211c9|OK  |    36KiB/s|/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/vsekidki

In [319]:
# for i in range(0, length):
#     # to rewrite all files delete "not"
words = []
no_definition = []
for i, word_page in enumerate(os.listdir('words_pages')):
    word_page = 'words_pages/' + word_page
    if i % 10 == 0:
        print(i, end=' ')
    with open(word_page, 'r') as word_page_file:
        word_page = word_page_file.read()
    soup = BeautifulSoup(word_page, "lxml")
    name = find_name(soup)
    definition = find_definition1(soup)
    word = {"name": name,
            "definition": definition}
    if word["name"].find("(Требуется значение слова)") == -1:
        words.append(word)
    else:
        no_definition.append(word)
dict_path = 'words_dict.npy'
np.save(dict_path, words)
print("*********\n ГОТОВО\n*********")

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 

/Users/lotfull/venvs/py36/lib/python3.6/site-packages/bs4/__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 *********
 ГОТОВО
*********


In [310]:
unique_links1 = set(map(lambda x: x.replace('https://vsekidki.ru/', ''), unique_links))
unique_links1

{'115-kak-v-armii-bystro-vyuchit-voinskie-zvaniya.html',
 '3935-outsayder.html',
 '9659-deshman.html',
 '3553-dereal.html',
 '651-fb-pervaya-krov.html',
 '578-haypovyy-hayp-haypim.html',
 '1244-boshki.html',
 '4218-ref.html',
 '131-brifovat.html',
 '275-obnovlennye-vse-kidki-godovschina-sayta.html',
 '1132-dvizh-dvizhuha.html',
 '969-tvzh.html',
 '9730-slaym.html',
 '939-pati.html',
 '3375-pucka.html',
 '507-merkantilnyy.html',
 '9644-kush.html',
 '9654-fotohanter.html',
 '501-armata.html',
 '848-bolty.html',
 '685-reper-oxxxymiron-miron-fedorov.html',
 '3566-sepese.html',
 '1213-fidbek.html',
 '1027-brazzers.html',
 '1303-armor.html',
 '400-kollaboraciya-supreme-s-lego.html',
 '574-lzya.html',
 '659-hard-layn-hl.html',
 '617-aoe-massiv.html',
 '9681-veykbord.html',
 '89-chto-sesh-mylo-so-stola-ili-hleb-s-parashi.html',
 '531-pistona-iz-bespredel-piston-i-moydodyr-kto-takoy.html',
 '127-sanskrin.html',
 '328-kek.html',
 '1071-enivey.html',
 '1329-krinzh.html',
 '944-flaska.html',
 '922

In [320]:
link_to_word = {
    link: word
    for link, word in zip(os.listdir('words_pages'), words)
}

In [333]:
words_clean = save_words_csv(words, vsekidki_appwords_csv)

In [425]:
words_clean.head()

,name,definition
0,"""Duck army"" ""Армия уток"" ""Утиная армия"" ""Утины...",Случайно наткнулся на интересный подарок в ВК ...
1,"""Спорт"" Гоша Рубчинский",A/W 2015 ГОША РУБЧИНКИЙ СПОРТ\nВсе коллекции: ...
2,"""Утка строящая глазки"" или ""Хитрая утка""","""Утка строящая глазки"" или ""Хитрая утка""\nОтно..."
3,#Mannequinchallenge,"Манекен Челлендж - вирусный тренд, в котором в..."
4,"#f57 (Море китов, тихий дом)",#f57 #f58 #тихийдом #морекитов #киты #китыумир...


In [340]:
words_clean.shape

(1869, 2)

In [390]:
len(link_to_cats)

1944

In [436]:
df = pd.DataFrame(link_to_word).transpose().sort_index()
df = df.reindex(index=df.index.to_series().str.split('-').str[0].astype(int).sort_values().index)
df.head(2)

,definition,name
13-huy.html,"теперь на заборе модно писать не только ""хуй"",...",Хуй жопа
15-pastoz-pastoznyy.html,"Что-то омерзительное или не хорошее, данным сл...",Пастоз (Пастозный)


In [ ]:
d = pd.DataFrame()
d['links'] = link_to_cats
d['cats'] = link_to_cats.values()
d = d.drop('links', axis=1)

In [437]:
d.head(2)

,cats
9764-deyliki.html,"[Модный словарь, Геймеры]"
9763-bed-bars.html,"[Модный словарь, Музыка]"


In [468]:
df.head(2)

,definition,name,cats
13-huy.html,"теперь на заборе модно писать не только ""хуй"",...",Хуй жопа,NaN
15-pastoz-pastoznyy.html,"Что-то омерзительное или не хорошее, данным сл...",Пастоз (Пастозный),NaN


In [463]:
nan = df.loc['13-huy.html']['cats']

In [473]:
type(df['cats'][21])

list

In [493]:
df['cats'] = d.cats
df['cats'] = df['cats'].fillna('Модный словарь')
df['cats'][df['cats'] == 'Модный словарь'] = 'Модный словарь'

In [494]:
df = df.reset_index()
df.head()

,index,definition,name,cats
0,13-huy.html,"теперь на заборе модно писать не только ""хуй"",...",Хуй жопа,Модный словарь
1,15-pastoz-pastoznyy.html,"Что-то омерзительное или не хорошее, данным сл...",Пастоз (Пастозный),Модный словарь
2,16-lyumpen-lumpen.html,"так то это Немодельное агенство, но в жизни мо...",Люмпен (Lumpen),Модный словарь
3,17-laksheri.html,"Английское словечко Luxury, что в переводе озн...",Лакшери,Модный словарь
4,18-mitboly.html,"А вот это уже классика, но пока что еще все ра...",Митболы,Модный словарь


In [501]:
def save_words_csv(words_df, file):
    words_df.replace('', np.nan, inplace=True)
    words_df.dropna(inplace=True)
    words_df = sort_df_by_name(words_df)
    words_df.to_csv(file, index=False)
    return words_df

save_words_csv(df, 'vse-kidki-2.csv')

,index,definition,name,cats
0,1379-duck-army-armiya-utok-utinaya-armiya-utin...,Случайно наткнулся на интересный подарок в ВК ...,"""Duck army"" ""Армия уток"" ""Утиная армия"" ""Утины...",[Сложный прекол!]
1,315-sport-gosha-rubchinskiy.html,A/W 2015 ГОША РУБЧИНКИЙ СПОРТ\nВсе коллекции: ...,"""Спорт"" Гоша Рубчинский",[Коллекции]
2,4158-utka-stroyaschaya-glazki-ili-hitraya-utka...,"""Утка строящая глазки"" или ""Хитрая утка""\nОтно...","""Утка строящая глазки"" или ""Хитрая утка""",[Сложный прекол!]
3,1461-mannequinchallenge.html,"Манекен Челлендж - вирусный тренд, в котором в...",#Mannequinchallenge,"[Модный словарь, Интернет]"
4,567-f57-more-kitov-tihiy-dom.html,#f57 #f58 #тихийдом #морекитов #киты #китыумир...,"#f57 (Море китов, тихий дом)",[Сложный прекол!]
5,8534-tbt.html,#TBT (ThrowBack To) - хештег который ставится ...,#tbt,"[Модный словарь, Интернет]"
6,4398-pahomov2018.html,#Пахомов2018\nКандидат от народа.\nКандидат дл...,#Пахомов2018,"[Модный словарь, Интернет, В жизни]"
7,4397-pahomov2018.html,Сергей Пахомов 2018\nКандидат от народа.\nКанд...,#Пахомов2018,[Сложный прекол!]
8,1675-skazochnoebali.html,"Свежий, сочный мемос, который пока еще только ...",#сказочноебали,[Сложный прекол!]
9,571--a-.html,/a/ что значит в кс? Символ /A/ обозначает хор...,/a/ в кс,"[Модный словарь, Геймеры]"


In [502]:
!open .

## Загрузить слова в df из CSV файлов

In [15]:
fullwords_df = pd.read_csv(vsekidki_fullwords_csv)
appwords_df = pd.read_csv(vsekidki_appwords_csv)

In [16]:
appwords_df.head(3)

,name,definition,type,group,examples,origin,hashtags,synonyms,id
0,#Mannequinchallenge,"Манекен Челлендж - вирусный тренд, в котором в...",NaN,NaN,NaN,"Самый крутой челлендж, по-моему мнению, на дан...",NaN,_,20001
1,#Пахомов2018,#Пахомов2018,NaN,NaN,NaN,Кандидат от народа.,NaN,_,20002
2,/a/ в кс,/a/ что значит в кс? Символ /A/ обозначает хор...,NaN,NaN,NaN,NaN,NaN,_,20003


In [17]:
fullwords_df.head(3)

,name,fulltext,id
0,#Mannequinchallenge,"['Манекен Челлендж - вирусный тренд, в котором...",20001
1,#Пахомов2018,"['#Пахомов2018', 'Кандидат от народа.', 'Канди...",20002
2,/a/ в кс,['/a/ что значит в кс? Символ /A/ обозначает х...,20003


In [74]:
print(fullwords_df['fulltext'][1188])

['Внимание!', 'Слово', 'Cиге', 'находится в категории "Объясните людям?" и требуется его значение. Если Вы знаете, откуда пошло "Cиге" или что это значит, пожалуйста напишите:', '+ Написать значение слова', "function openbox(id){\n    display = document.getElementById(id).style.display;\n\n    if(display=='none'){\n       document.getElementById(id).style.display='block';\n    }else{\n       document.getElementById(id).style.display='none';\n    }\n}"]


## Сохранить dataFrame в CSV файлы

In [14]:
appwords_df.to_csv(vsekidki_appwords_csv, index=False)
fullwords_df.to_csv(vsekidki_fullwords_csv, index=False)

## Добавил столбец id

In [13]:
appwords_df["id"] = appwords_df.index + 1 + id_offset
appwords_df.head(3)
fullwords_df["id"] = fullwords_df.index + 1 + id_offset
fullwords_df.head(3)

,name,fulltext,id
0,#Mannequinchallenge,"['Манекен Челлендж - вирусный тренд, в котором...",20001
1,#Пахомов2018,"['#Пахомов2018', 'Кандидат от народа.', 'Канди...",20002
2,/a/ в кс,['/a/ что значит в кс? Символ /A/ обозначает х...,20003
